## MERGE INTO

MERGE INTO is a powerful SQL statement used to synchronize a Delta Lake table with updates, inserts, or deletes from another dataset—all in one go.

This is often called "upsert", and it's a very common way to:
- Keep a table updated with incoming changes
- Avoid writing separate UPDATE and INSERT statements
- Do this atomically (all-or-nothing)

In [0]:
DROP TABLE IF EXISTS target_customers;
CREATE TABLE target_customers (
  customer_id INT,
  customer_name STRING,
  country STRING
) USING DELTA;

INSERT INTO target_customers VALUES
(1, 'Alice', 'USA'),
(2, 'Bob', 'UK'),
(3, 'Charlie', 'India');

In [0]:
SELECT * FROM target_customers;

### Create Source DataFrame (or Table)

In [0]:
%python
data = [
    (2, "Robert", "UK"),       # Update Bob to Robert
    (3, "Charlie", "Canada"),  # Update country to Canada
    (4, "Diana", "Australia")  # New customer
]

columns = ["customer_id", "customer_name", "country"]
source_df = spark.createDataFrame(data, columns)

source_df.createOrReplaceTempView("source_customers")
spark.sql("SELECT * FROM source_customers").display()

### Perform MERGE INTO

In [0]:
MERGE INTO target_customers AS target
USING source_customers AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
  UPDATE SET
    target.customer_name = source.customer_name,
    target.country = source.country
WHEN NOT MATCHED THEN
  INSERT (
    customer_id,
    customer_name,
    country
  )
  VALUES (
    source.customer_id,
    source.customer_name,
    source.country
  );


In [0]:
SELECT * FROM target_customers;

### Important Notes
`MERGE INTO` requires the target table to be Delta format.

You can have multiple clauses:

- `WHEN MATCHED AND <condition> THEN DELETE`
- `WHEN MATCHED THEN UPDATE`
- `WHEN NOT MATCHED THEN INSERT`

You can also merge using another Delta table instead of a DataFrame.

### Example with Delete Clause

In [0]:
MERGE INTO target_customers AS target
USING source_customers AS source
ON target.customer_id = source.customer_id
WHEN MATCHED AND source.country = 'Australia' THEN
  DELETE
WHEN MATCHED THEN
  UPDATE SET
    target.customer_name = source.customer_name,
    target.country = source.country
WHEN NOT MATCHED THEN
  INSERT (
    customer_id,
    customer_name,
    country
  )
  VALUES (
    source.customer_id,
    source.customer_name,
    source.country
  );

In [0]:
SELECT * FROM target_customers;

### Re-create all table

In [0]:
DROP TABLE IF EXISTS target_customers;

CREATE TABLE target_customers (
  customer_id INT,
  customer_name STRING,
  country STRING
) USING DELTA;

INSERT INTO target_customers VALUES
(1, 'Alice', 'USA'),
(2, 'Bob', 'UK'),
(3, 'Charlie', 'India'),
(5, 'Edward', 'USA');

SELECT * FROM target_customers ORDER BY customer_id;

In [0]:
%python

data = [
    (2, 'Robert', 'UK'),       # UPDATE
    (3, 'Charlie', 'Canada'),  # UPDATE
    (4, 'Diana', 'Australia'), # INSERT
    (5, 'Edward', 'DELETE')    # Will trigger DELETE
]

columns = ["customer_id", "customer_name", "country"]

source_df = spark.createDataFrame(data, columns)

# Register temp view
source_df.createOrReplaceTempView("source_customers")
source_df.show()

## MEGRE INTO Example with PySpark API

In [0]:
%py
from delta.tables import DeltaTable

# Load the target Delta table
delta_table = DeltaTable.forName(spark, "target_customers")

# Perform merge
delta_table.alias("target").merge(
    source_df.alias("source"),
    "target.customer_id = source.customer_id"
).whenMatchedUpdate(
    condition = "source.country != 'DELETE'",
    set = {
        "customer_name": "source.customer_name",
        "country": "source.country"
    }
).whenMatchedDelete(
    condition = "source.country = 'DELETE'"
).whenNotMatchedInsert(
    values = {
        "customer_id": "source.customer_id",
        "customer_name": "source.customer_name",
        "country": "source.country"
    }
).execute()


In [0]:
SELECT * FROM target_customers ORDER BY customer_id;